<a href="https://colab.research.google.com/github/kristinoktvn/Data-Mining-4611-11271/blob/main/KNN_Gibran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [1]:
%pip install ekphrasis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 581.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 3.1 MB/s eta 0:00:00


In [2]:
# Import library Ekphrasis
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    annotate={"hashtag","allcaps","elongated","repeated",'emphasis','censored'},
    fix_html=True,

    # corpus from which the word statistics are going to be used
    # for word segmentation
    segmenter="twitter",

    # corpus from which the word statistics are going to be used
    # for spell correction
    corrector="twitter",

    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,

    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [3]:
import re
import string
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# Import Dataset by Drive

In [4]:
# To Connecting GDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Read the dataset
data = pd.read_csv('/content/drive/MyDrive/FILE JOKI/M.ZIDNI ILMAN_A12.2019.06270/Bahan_Dataset/data fix.csv')
data

FileNotFoundError: ignored

In [ ]:
# Delete the username column
data = data.drop(['Username'], axis=1)
data

In [ ]:
# Delete the date column
data = data.drop(['Date'], axis=1)
data

# Preprocessing Tweet

### Cleaning Data
#### Menghapus karatakter yang tidak diperlukan seperti hastag, tag, username, angka, link/url, dan elemen lainnya

In [ ]:
# Cleaning data with a text processor from the Ekphrasis library
def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

# Replaces non-ASCII characters
def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

# Removes excess spaces
def remove_space(text):
    return " ".join(text.split())

# Removed excess emojis
def remove_emoji(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

# Removes special characters ("\t", "\n", "\u")
def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

# Replace one or more space characters with a single space using regex
def remove_tab2(text):
    return re.sub('\s+',' ',text)

# Removed RT/Retweet terms
def remove_rt(text):
    return text.replace('RT'," ")

# Deleting mentions
def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

# Deleting url
def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

# Delete single characters
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

# Replace dashes with spaces
def change_stripe(text):
    return text.replace('-'," ")

# Remove punctuation
def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text)

### Case Folding
#### Mengubah semua huruf menjadi lower atau huruf kecil

In [ ]:
def lower(text):
    return text.lower()

In [ ]:
final_string = []
for text in data['Komen'].values:
    EachReviewText = ""
    proc = bersih_data(text)
    proc = remove_rt(proc)
    proc = lower(proc)
    proc = change_stripe(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_single_char(proc)
    proc = remove_punctuation(proc)
    proc = remove_space(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)

In [ ]:
# View the results of the data that has been cleaned
data["step01"] = final_string
data

In [ ]:
# View data information
data.info()

### Tokenizing
#### Memisahkan kata

In [ ]:
# Import library nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
# Tokenizing
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [ ]:
# View the results of data that has been separated by words
data['tokens'] = data['step01'].apply(word_tokenize_wrapper)
data

### Normalisasi
#### Penggantian kata-kata dalam teks berdasarkan kamus perbaikan kata

In [ ]:
# Calls the word repair dictionary file
normalized_word = pd.read_excel("/content/drive/MyDrive/FILE JOKI/M.ZIDNI ILMAN_A12.2019.06270/Bahan_Dataset/kamus perbaikan kata.xlsx")

normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

data['tokens_perbaikan'] = data['tokens'].apply(normalized_term)

data

### Stopword Removal
#### Mengapus kata yang bersifat umum seperti (aku, kamu, di, yang, dll)

In [ ]:
# Download a collection of stopwords provided by NLTK
from nltk.corpus import stopwords
nltk.download("stopwords")

In [ ]:
# Cleans text from words identified as stopwords in Indonesian
stopword = nltk.corpus.stopwords.words('indonesian')

def remove_stopwords(text):
  text = [word for word in text if word not in stopword]
  return text

data['Stop_removal'] = data['tokens_perbaikan'].apply(lambda x: remove_stopwords(x))
data

In [ ]:
# Converts a list of each row in the 'Stop_removal' column to single text separated by spaces
i=0
final_string_tokens = []
for text in data['Stop_removal'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [ ]:
# View data results
data["step02"] = final_string_tokens
data

### Stemming
#### Mengubah kata yang memiliki imbuhan menjadi kata dasar

In [ ]:
# Installing the Sastrawi Module
!pip install sastrawi

In [ ]:
# Importing the StemmerFactory module from Sastrawi which is a library for doing stemming in Indonesian
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
# Stemming Process
final_string = []
s = ""
total_sentences = len(data["step02"].values)  # Mengetahui total kalimat dalam dataset
for idx, sentence in enumerate(data["step02"].values, start=1):
    print(f"Melakukan stemming pada kalimat ke-{idx} dari total {total_sentences} kalimat")

    filteredSentence = []
    EachReviewText = ""
    s = (stemmer.stem(sentence))
    filteredSentence.append(s)

    EachReviewText = ' '.join(filteredSentence)
    final_string.append(EachReviewText)

print("Proses stemming selesai.")

In [ ]:
# View data results
data["ProcessedText"] = final_string
data

In [ ]:
# Save the dataframe in CSV format
data.to_csv('hasilpreprocessing.csv',sep=";")

### Pembobotan Kata
#### Mengetahui seberapa sering muncul kata yang ada pada komentar

### Implementasi KNN

In [ ]:
# Import Library
import pandas as pd
import numpy as np

In [ ]:
# Import the dataframe that has previously been word cleaned
dataset = pd.read_csv('hasilpreprocessing.csv',sep=";")
dataset

In [ ]:
# View dataset info
dataset.info()

In [ ]:
# Ensures that the content of the 'ProcessedText' column is considered a string data type
dataset_feature = dataset['ProcessedText'].astype(str)
dataset_feature

In [ ]:
# Labels or targets when performing machine learning tasks
dataset_label = dataset['Label']
dataset_label

In [ ]:
# Import Library
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Counts the number of unique occurrences of a value in a column in a DataFrame
dataset_label.value_counts()

# Visulisasi Data

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

value_counts = dataset['Label'].value_counts()

plt.figure(figsize=(6, 6))
plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=90, colors=['#66b3ff','#99ff99'])
plt.title('Chart Label')
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split

In [ ]:
# Dividing the dataset between training and testing
train_x, test_x, train_label, test_label = train_test_split(dataset_feature, dataset_label, test_size = 0.1, random_state=0)

In [ ]:
# imported the TfidfVectorizer module from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

# Inisialisasi objek TfidfVectorizer
Tfidf_Vectorizer = TfidfVectorizer()

# Perform transformation and formation of TF-IDF feature vectors from text data
tfidf_matrix = tfidf_vectorizer.fit_transform(dataset)



In [ ]:
# Displays the shape of the TF-IDF feature vector
print("Shape of TF-IDF Matrix:", tfidf_matrix.shape)

# Returns a TF-IDF feature vector
print("TF-IDF Matrix:")
print(tfidf_matrix.toarray())

# Implementasi KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from sklearn.pipeline import Pipeline
classifier_knn = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('model', KNeighborsClassifier()),
])

In [ ]:
hasil = np.arange(1, 30, 2, dtype=int)
hasil

In [ ]:
parameters_knn = {
               'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4),(2,3),(2,4),(3,4)],
               'tfidf__use_idf': (True, False),
               'model__n_neighbors': np.arange(1, 30, 2, dtype=int)
}

In [ ]:
classifier_knn = GridSearchCV(classifier_knn, parameters_knn, cv = 3, n_jobs=-1)
classifier_knn.fit(train_x, train_label.ravel())

In [ ]:
y_pred_knn_train = classifier_knn.predict(train_x)
accuracy_knn_train = accuracy_score(train_label, y_pred_knn_train)
print("Accuracy Training set: ", accuracy_knn_train)

y_pred_knn_test = classifier_knn.predict(test_x)
accuracy_knn_test = accuracy_score(test_label, y_pred_knn_test)
print("Accuracy Test set: ", accuracy_knn_test)

In [ ]:
recall_knn_train = recall_score(train_label, y_pred_knn_train, average='weighted')
print("Recall Training set: ", recall_knn_train)

recall_knn_test = recall_score(test_label, y_pred_knn_test, average='weighted')
print("Recall Test set: ", recall_knn_test)

In [ ]:
precision_knn_train = precision_score(train_label, y_pred_knn_train, average='weighted')
print("Precision Training set: ", precision_knn_train)

precision_knn_test = precision_score(test_label, y_pred_knn_test, average='weighted')
print("Precision Test set: ", precision_knn_test)

In [ ]:
f1_knn_train = f1_score(train_label, y_pred_knn_train, average='weighted')
print("F1 Training set: ", f1_knn_train)

f1_knn_test = f1_score(test_label, y_pred_knn_test, average='weighted')
print("F1 Test set: ", f1_knn_test)

In [ ]:
from sklearn.metrics import confusion_matrix
sns.heatmap(confusion_matrix(test_label, y_pred_knn_test), annot=True, cmap = 'viridis', fmt='.0f')
plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)
plt.show()

In [ ]:
classifier_knn.best_estimator_

In [ ]:
for param_name in sorted(parameters_knn.keys()):
    print("%s: %r" % (param_name, classifier_knn.best_params_[param_name]))

##Wordcloud

In [ ]:
!pip install wordcloud

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, ImageColorGenerator
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
#Word Cloud Teks
f = open("/content/drive/MyDrive/FILE JOKI/M.ZIDNI ILMAN_A12.2019.06270/Bahan_Dataset/Text Wordcloud.txt", "r")

isi_text = f.read()

In [ ]:
#baca data
print(isi_text)

In [ ]:
wordcloud = WordCloud(width=1600, height=800, max_font_size=200, max_words=90, colormap='Set2', background_color='black')

In [ ]:
wordcloud.generate(isi_text)

In [ ]:
plt.figure(figsize=(12,10))

plt.imshow(wordcloud, interpolation='bilinear')
plt.savefig('wordcloud_positif.png')
plt.axis("off")

plt.show()

In [ ]:
text_dictionary = wordcloud.process_text(isi_text)
word_freq_pos = []

word_freq_pos = {k: v for k, v in sorted(text_dictionary.items(),reverse=True, key=lambda item: item[1])}

print(list(word_freq_pos.items())[:10])